In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime
from itertools import product
from model import NeuralNet
from simulated_annealing import SA, plot_histories
from scipy.special import softmax 
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from tqdm.notebook import tqdm

np.set_printoptions(precision=4, suppress=True)
np.random.seed(0)

In [ ]:
data_attrs = load_iris()
data_attrs.keys()

data = data_attrs.data
y = data_attrs.target
print(data.shape, y.shape)

data_norm = (data - data.mean(axis=0)) / data.std(axis=0)

In [ ]:
model = NeuralNet(data=data_norm, y=y, hidden_units=2)    # Criando o modelo de Rede Neural
num_iter = 5                                              # Número de execuções para cada algoritmo

print(f'Quantidade de parâmetros da rede: {model.size(model.sample_weights())}')

In [ ]:
# Parâmetros da Rede Neural a partir do SGD
lbfgs_loss, lbfgs_exec_time = np.zeros(num_iter), []

for i in range(num_iter):
    mlp_classifier = MLPClassifier(hidden_layer_sizes=(2,), solver='lbfgs', alpha=0.0, max_iter=2000, max_fun=50000, verbose=False)
    start = datetime.now()
    mlp_classifier.fit(data_norm, y)
    lbfgs_exec_time.append(datetime.now() - start)
    
    p = mlp_classifier.predict_proba(data_norm)
    lbfgs_loss[i] = model.cross_entropy(y, p)

In [ ]:
# Parâmetros da Rede Neural a partir do Simulated Annealing
results = []

N_options = [int(1e4), int(1e5)]
T0_options = [1.]
K_options =  [6]

start_datetime = str(datetime.now()).split('.')[0].replace(' ', '_').replace(':', '').replace('-', '')

for N, T0, K in product(N_options, T0_options, K_options):
    histories = {}
    eps = 0.01
    for i in tqdm(range(num_iter), desc=f'T0 = {T0}, K = {K}, N = {N}'):
        J_min, x_min, history, exec_time = SA(model, N=N, K=K, T0=T0, eps=eps)
        results.append([N, K, T0, eps, J_min, x_min, exec_time])
        if i == 0: histories['T'] = history[1, :]
        histories[f'iter {i+1}'] = history[0, :]
    hists = pd.DataFrame(histories)
    try: os.mkdir(f".\\results\\Iris_{start_datetime}")
    except: pass
    hists.to_pickle(f".\\results\\Iris_{start_datetime}\\histories_T0-{T0}_K-{K}_N-{N}.pkl")
        
df = pd.DataFrame(results, columns=['N', 'K', 'T0', 'eps', 'J mínimo', 'x mínimo', 'Tempo de Execução'])
df.to_pickle(f".\\results\\Iris_{start_datetime}\\results.pkl")

In [ ]:
# start_datetime = '20220507_143854'

df = pd.read_pickle(f'.\\results\\Iris_{start_datetime}\\results.pkl')

# L-BFGS
print(f'L-BFGS, J_mínimo: {np.round(lbfgs_loss.mean(), 4)} +/- {np.round(lbfgs_loss.std(), 4)}, Tempo de execução médio: {np.mean(lbfgs_exec_time)}')
print()

# Simulated Annealing
for N, T0, K in product(N_options, T0_options, K_options):
    # Plot optimization history
    histories = pd.read_pickle(f'.\\results\\Iris_{start_datetime}\\histories_T0-{T0}_K-{K}_N-{N}.pkl')
    plot_histories(histories, T0, K, N, savefig=True, dt=start_datetime)

    # Plot results
    filter_ = (df['T0'] == T0) & (df['K'] == K) & (df['N'] == N)
    print(f'N: {N}', end=', ')
    J_mean, J_std = df[filter_]['J mínimo'].mean(), df[filter_]['J mínimo'].std()
    t_mean = df[filter_]['Tempo de Execução'].mean().to_pytimedelta()

    print(f"J_mínimo: {np.round(J_mean, 4)} +/- {np.round(J_std, 4)}", end=', ')
    print(f"Tempo de execução médio: {t_mean}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

ax.boxplot([lbfgs_loss.tolist()] + [df[df['N'] == N]['J mínimo'] for N in N_options], labels=['L-BFGS'] + N_options)
ax.grid()
ax.set_ylabel('J mínimo')
ax.set_xlabel('N (Número de iterações por temperatura)')
plt.show()

In [ ]:
idx = num_iter * (len(N_options) - 1) + df[df['N'] == N_options[-1]]['J mínimo'].argmin()
x_min = df.iloc[idx, 5]
df.iloc[idx, :]

In [ ]:
print(np.hstack([model(data_norm, x_min)[:50], model(data_norm, x_min)[50:100], model(data_norm, x_min)[100:]]))

In [ ]:
bests = df.iloc[[i*num_iter + df[df['N'] == N]['J mínimo'].argmin() for i, N in enumerate(N_options)], :]
bests

In [ ]:
x_min